<a href="https://colab.research.google.com/github/arthur-kt/arthur-kt/blob/main/PPP_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fredapi import Fred
from pppfy.converter import Converter
import pandas as pd

# Inicialize o conversor do PPPfy e configure a chave do FRED
c = Converter()
fred = Fred(api_key='11cb1e53b57458acaddd29a16e0efe1b')

# Obtenha o CPI dos EUA para ajustar pela inflação
cpi_data = fred.get_series('CPIAUCSL')
cpi_df = cpi_data.to_frame(name='CPI')
cpi_df.index = pd.to_datetime(cpi_df.index)
cpi_df_ano = cpi_df.resample('Y').mean()  # Média anual do CPI

# CPI atual para trazer o valor a 2024
cpi_atual = cpi_df_ano.loc['2024'].values[0]

# Dados de exemplo
dados = {
    'País': ['BR', 'DE', 'IN'],  # Usando códigos ISO2 dos países
    'Ano': [2010, 2015, 2012],
    'Gasto_Local': [5000, 7000, 3000]  # Gasto em moeda local
}

df = pd.DataFrame(dados)

# Função para converter gasto local em dólar PPP e ajustar pela inflação dos EUA
def converter_para_dolar_corrente(row):
    # Converta o gasto local para dólares ajustados pelo PPP usando pppfy
    ppp_adjusted = c.get_price_mapping(
        source_country=row['País'],
        source_price=row['Gasto_Local'],
        destination_country="US",
        year=row['Ano']
    )
    gasto_em_dolar_ppp = ppp_adjusted["ppp_adjusted_local_price"]

    # Ajuste pela inflação dos EUA
    cpi_ano_referencia = cpi_df_ano.loc[str(row['Ano'])].values[0]
    gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_ano_referencia)

    return gasto_ajustado

# Aplicar a função para cada linha no DataFrame
df['Gasto_Ajustado_Dolar'] = df.apply(converter_para_dolar_corrente, axis=1)

# Resultado final
print(df[['País', 'Ano', 'Gasto_Local', 'Gasto_Ajustado_Dolar']])
